## Census Data Cleaning

In [1]:
import pandas as pd

In [2]:
census_raw_file = "data/Census_data_raw.csv"
raw_census = pd.read_csv(census_raw_file, dtype=str)
raw_census[['Tract_name', 'County_name', 'State_name']] = raw_census["DETAILS"].str.split(';', expand=True)
raw_census.head()

,DETAILS,MALES_UNDER5,FEMALES_UNDER5,TOTPOP,WHITE,BLACK,ASIAN,HISPANIC,BELOW_POVERTY_LINE,MEDIAN_INCOME,...,SAME_HOUSE_AS_LAST_YEAR,POP_OVER25,LESS_THAN_HS,BACHELOR_OR_GREATER,STATE,COUNTY,TRACT,Tract_name,County_name,State_name
0,Census Tract 1; Adams County; Illinois,112,121,4509,4028,263,60,0,440,61595,...,3827,3443,341,810,17,001,000100,Census Tract 1,Adams County,Illinois
1,Census Tract 2.01; Adams County; Illinois,49,61,1968,1777,134,9,32,192,44583,...,1857,1455,141,390,17,001,000201,Census Tract 2.01,Adams County,Illinois
2,Census Tract 2.02; Adams County; Illinois,54,31,2473,2171,110,35,51,257,66472,...,2025,1491,106,470,17,001,000202,Census Tract 2.02,Adams County,Illinois
3,Census Tract 4; Adams County; Illinois,145,63,3428,2829,353,0,54,718,42413,...,2761,2441,275,232,17,001,000400,Census Tract 4,Adams County,Illinois
4,Census Tract 5; Adams County; Illinois,63,82,1985,1727,104,21,241,499,39167,...,1577,1360,139,183,17,001,000500,Census Tract 5,Adams County,Illinois


In [3]:
# remove rows with tracts that do not have any population
raw_census = raw_census[raw_census["TOTPOP"].astype(int) > 0]

# raw data has -666666666 as the median income if there is no figure available, set this to mean of state"
mean_median_income = int(raw_census["MEDIAN_INCOME"][raw_census["MEDIAN_INCOME"].astype(int) > 0].astype(int).mean())
raw_census.loc[raw_census["MEDIAN_INCOME"].astype(int) <= 0, "MEDIAN_INCOME"] = str(mean_median_income)

# median income bins
med_income_quantiles = raw_census['MEDIAN_INCOME'].astype(int).quantile([0, 1/3, 2/3, 1])
labels = ['low', 'medium', 'high']

# drop the tracts that do not have any population


In [11]:
CensusData = pd.DataFrame()

In [16]:
CensusData["state_name"] = raw_census["State_name"]
CensusData["state_code"] = raw_census["STATE"].astype(str)
CensusData["county_name"] = raw_census["County_name"]
CensusData["county_code"] = raw_census["COUNTY"].astype(str)
CensusData["tract_name"] = raw_census["Tract_name"]
CensusData["tract_code"] = raw_census["TRACT"].astype(str)
CensusData["tot_pop"] = raw_census["TOTPOP"].astype(int)
CensusData["pop_under5"] = raw_census["MALES_UNDER5"].astype(int) + raw_census["FEMALES_UNDER5"].astype(int)
CensusData["homeowner_rate"] = round(raw_census["HOMEOWNER_OCCUPIED_HOUSES"].astype(int)/raw_census["TOTAL_OCCUPIED_HOUSES"].astype(int), 4)
CensusData["less_than_hs_rate"] = round(raw_census["LESS_THAN_HS"].astype(int)/raw_census["POP_OVER25"].astype(int), 4)
CensusData["higher_education_rate"] = round(raw_census["BACHELOR_OR_GREATER"].astype(int)/raw_census["POP_OVER25"].astype(int), 4)
CensusData["below_poverty_rate"] = round(raw_census["BELOW_POVERTY_LINE"].astype(int)/raw_census["TOTPOP"].astype(int), 4)
CensusData["income_cat"] = pd.cut(raw_census['MEDIAN_INCOME'].astype(int), bins=med_income_quantiles, labels=labels)

def is_majority(population, total_population):
    proportion = population / total_population
    return 1 if proportion > 0.5 else 0

CensusData["majority_white"] = raw_census.apply(lambda row: is_majority(int(row["WHITE"]), int(row["TOTPOP"])), axis=1)
CensusData["majority_black"] = raw_census.apply(lambda row: is_majority(int(row["BLACK"]), int(row["TOTPOP"])), axis=1)
CensusData["majority_asian"] = raw_census.apply(lambda row: is_majority(int(row["ASIAN"]), int(row["TOTPOP"])), axis=1)
CensusData["majority_hispanic"] = raw_census.apply(lambda row: is_majority(int(row["HISPANIC"]), int(row["TOTPOP"])), axis=1)



In [17]:
CensusData.head()

,state_name,state_code,county_name,county_code,tract_name,tract_code,tot_pop,pop_under5,homeowner_rate,less_than_hs_rate,higher_education_rate,below_poverty_rate,income_cat,majority_white,majority_black,majority_asian,majority_hispanic
0,Illinois,17,Adams County,001,Census Tract 1,000100,4509,233,0.8645,0.0990,0.2353,0.0976,low,1,0,0,0
1,Illinois,17,Adams County,001,Census Tract 2.01,000201,1968,110,0.7971,0.0969,0.2680,0.0976,low,1,0,0,0
2,Illinois,17,Adams County,001,Census Tract 2.02,000202,2473,85,0.6852,0.0711,0.3152,0.1039,medium,1,0,0,0
3,Illinois,17,Adams County,001,Census Tract 4,000400,3428,208,0.5094,0.1127,0.0950,0.2095,low,1,0,0,0
4,Illinois,17,Adams County,001,Census Tract 5,000500,1985,145,0.4804,0.1022,0.1346,0.2514,low,1,0,0,0


In [18]:
file_path = "data/Census_data.csv"
CensusData.to_csv(file_path, index=False)